<a href="https://colab.research.google.com/github/GProvan/CS4705/blob/main/_notebooks/2020-10-26-02-RNN-Basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNN Basic
> In this post, We will briefly cover the Recurrent Neural Network (RNN) and its implementation in tensorflow.

- toc: true
- badges: true
- comments: true
- author: Chanseok Kang
- categories: [Python, Deep_Learning, Tensorflow-Keras]
- image: images/simpleRNN.png

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

print('Tensorflow: {}'.format(tf.__version__))

plt.rcParams['figure.figsize'] = (16, 10)
plt.rc('font', size=15)

Tensorflow: 2.17.0


## RNN in Tensorflow
Recurrent Neural Network (RNN for short) is the neural network that has backward stream into input node. Simple notation is expressed like this,

![RNN](https://github.com/goodboychan/goodboychan.github.io/blob/main/_notebooks/image/simpleRNN.png?raw=1)

And it is implemented in Tensorflow (of course, it can be easily used with tensorflow keras). There are two implementation approaches,

- Using basic cell (`SimpleRNNCell`) and merge it with multiple elements to build complex model like Long Short Term Mermory (LSTM) or Gated Recurrent Unit (GRU)

```python
cell = tf.keras.layers.SimpleRNNCell(units=hidden_size)
rnn = tf.keras.layers.RNN(cell, return_sequences=True, return_state=True)
outputs, states = rnn(X_data)
```

- Using RNN Layer (`SimpleRNN`)

```python
rnn = tf.keras.layers.SimpleRNN(units=hidden_size, return_sequences=True,
                                return_state=True)
outputs, states = rnn(X_data)
```

## Simple RNN Cell
Let's look at more closely with example. Suppose that we want to train RNN that can generate the word "hello". Of course, the model cannot interpret word itself, so it is required to convert it with one-hot encoding. Currently, "Hello" is the word with 4 characters, "h", "e", "l", "o". So we can define each character with one-hot expression.

In [2]:
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

In tensorflow, to insert this data, we need to reshape like this order:

$$ (\text{batch_size}, \text{sequence_length}, \text{sequence_width}) $$

If we want to insert one-hot encoded `h` in RNN as an input, we can make it with numpy array,

In [3]:
X_data = np.array([[h]], dtype=np.float32)
X_data.shape

(1, 1, 4)

Then, we can make RNN cell with hidden_size of 2.

In [4]:
hidden_size = 2
cell = tf.keras.layers.SimpleRNNCell(units=hidden_size)
rnn = tf.keras.layers.RNN(cell, return_sequences=True, return_state=True)
outputs, states = rnn(X_data)

print("X_data: {}, shape: {}".format(X_data, X_data.shape))
print("output: {}, shape: {}".format(outputs, outputs.shape))
print("states: {}, shape: {}".format(states, states.shape))

X_data: [[[1. 0. 0. 0.]]], shape: (1, 1, 4)
output: [[[-0.56661063  0.724539  ]]], shape: (1, 1, 2)
states: [[-0.56661063  0.724539  ]], shape: (1, 2)


Mentioned before, we can also build it with layer API.

In [5]:
rnn_2 = tf.keras.layers.SimpleRNN(units=hidden_size, return_sequences=True, return_state=True)
outputs, states = rnn_2(X_data)

print("X_data: {}, shape: {}".format(X_data, X_data.shape))
print("output: {}, shape: {}".format(outputs, outputs.shape))
print("states: {}, shape: {}".format(states, states.shape))

X_data: [[[1. 0. 0. 0.]]], shape: (1, 1, 4)
output: [[[ 0.74435496 -0.29972216]]], shape: (1, 1, 2)
states: [[ 0.74435496 -0.29972216]], shape: (1, 2)


## Unfolding multiple sequences
But in above example, the output is meaningless. RNN is neural network for sequence data, but we just insert 1-sequence data. In order to understand the RNN operation, we need to insert multiple-length sequences.

![rnn_example](https://github.com/goodboychan/goodboychan.github.io/blob/main/_notebooks/image/rnn_example1.png?raw=1)

In this case, we will insert 5-length sequences. So input data shape will be like this,

$$ (1, 5, 4) $$

Currently, we just use 1 batch, and 5-length sequence, and our data have 4 width after one-hot encoding. If the hidden size is 2, then the output shape is aligned to hidden_size.

Anyway, Our input data will be like this,

In [6]:
X_data = np.array([[h, e, l, l, o]], dtype=np.float32)
X_data.shape

(1, 5, 4)

Same as before, we can build RNN cell.

In [7]:
cell = tf.keras.layers.SimpleRNNCell(units=hidden_size)
rnn = tf.keras.layers.RNN(cell, return_sequences=True, return_state=True)
outputs, states = rnn(X_data)

print("X_data: {}, shape: {}\n".format(X_data, X_data.shape))
print("output: {}, shape: {}\n".format(outputs, outputs.shape))
print("states: {}, shape: {}".format(states, states.shape))

X_data: [[[1. 0. 0. 0.]
  [0. 1. 0. 0.]
  [0. 0. 1. 0.]
  [0. 0. 1. 0.]
  [0. 0. 0. 1.]]], shape: (1, 5, 4)

output: [[[ 0.27097753  0.7526068 ]
  [-0.18402556  0.3310005 ]
  [ 0.39019778  0.5721705 ]
  [ 0.13431062  0.836263  ]
  [ 0.7391487   0.16480744]]], shape: (1, 5, 2)

states: [[0.7391487  0.16480744]], shape: (1, 2)


Output contains each hidden states while input data is processed in RNN cell. So the the last element in output is the same as states, since it is the latest states after process.

## Batching Input
Of course, we can extend the dataset with batch size.

![example2](https://github.com/goodboychan/goodboychan.github.io/blob/main/_notebooks/image/rnn_example2.png?raw=1)

Suppose that we have 3 batch data as inputs, and our input data is `hello`, `eolll`, and `lleel`. Don't worry about complex shape dataset. All the process is the same as before. All you need to do is to make the input dataset with appropriate shape. So in this case, we need to make (3, 5, 4)-shaped input.



In [8]:
X_data = np.array([[h, e, l, l, o],
                   [e, o, l, l, l],
                   [l, l, e, e, l]], dtype=np.float32)
X_data.shape

(3, 5, 4)

In [9]:
cell = tf.keras.layers.SimpleRNNCell(units=hidden_size)
rnn = tf.keras.layers.RNN(cell, return_sequences=True, return_state=True)
outputs, states = rnn(X_data)

print("X_data: {}, shape: {}\n".format(X_data, X_data.shape))
print("output: {}, shape: {}\n".format(outputs, outputs.shape))
print("states: {}, shape: {}".format(states, states.shape))

X_data: [[[1. 0. 0. 0.]
  [0. 1. 0. 0.]
  [0. 0. 1. 0.]
  [0. 0. 1. 0.]
  [0. 0. 0. 1.]]

 [[0. 1. 0. 0.]
  [0. 0. 0. 1.]
  [0. 0. 1. 0.]
  [0. 0. 1. 0.]
  [0. 0. 1. 0.]]

 [[0. 0. 1. 0.]
  [0. 0. 1. 0.]
  [0. 1. 0. 0.]
  [0. 1. 0. 0.]
  [0. 0. 1. 0.]]], shape: (3, 5, 4)

output: [[[-0.5156224   0.35898373]
  [ 0.15212457  0.02775879]
  [-0.39157686 -0.02593066]
  [ 0.12581399 -0.10759626]
  [ 0.34998876  0.616439  ]]

 [[-0.25226462  0.4643481 ]
  [ 0.5304369   0.07158946]
  [-0.65902925  0.02489586]
  [ 0.3567318  -0.21960385]
  [-0.5012186   0.25826764]]

 [[-0.25372475 -0.03654187]
  [-0.00449121 -0.06364252]
  [-0.23339728  0.5105023 ]
  [-0.15627259 -0.04941762]
  [-0.09546486 -0.02720663]]], shape: (3, 5, 2)

states: [[ 0.34998876  0.616439  ]
 [-0.5012186   0.25826764]
 [-0.09546486 -0.02720663]], shape: (3, 2)


Mentioned before, output contains each hidden states while processing the input data. Each hidden state affects the next hidden state and so on. Then the last hidden state will be states that outputs from RNN cell.

## Summary
Through this post, we covered simple usage of RNN cell and layers in Tensorflow. We just define the dataset with appropriate format, then the RNN layer can generates hidden states for the output that affects to next state.